In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'

In [6]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

***EDA***

In [7]:
#EDA 함수 정의
def basic_eda(df, name):
    print(f"========== {name} ==========")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("\n[INFO]")
    print(df.info())
    print("\n[Missing Values]")
    print(df.isnull().sum())
    print("\n[Descriptive Statistics]")
    print(df.describe(include='all'))
    print("\n[Head]")
    print(df.head())
    print("\n[Tail]")
    print(df.tail())
    print("\n===============================\n")

In [8]:
print("***** Sales Data EDA *****")
basic_eda(df_sales, "Sales Data")

***** Sales Data EDA *****
========== Sales Data ==========
Shape: (668111, 12)
Columns: ['회원번호', '회원상태', '구매수량', '구매금액', '주문일시', '배송시작일', '배송완료일', '사용 적립금', '사용 포인트 네이버', '주문취소여부', '주문시간', '제품번호']

[INFO]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668111 entries, 0 to 668110
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   회원번호        668111 non-null  int64  
 1   회원상태        668109 non-null  object 
 2   구매수량        668111 non-null  float64
 3   구매금액        668111 non-null  int64  
 4   주문일시        668111 non-null  object 
 5   배송시작일       637476 non-null  object 
 6   배송완료일       637476 non-null  object 
 7   사용 적립금      668111 non-null  int64  
 8   사용 포인트 네이버  668111 non-null  int64  
 9   주문취소여부      30635 non-null   object 
 10  주문시간        668111 non-null  object 
 11  제품번호        668111 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 61.2+ MB
None

[Missing Values]
회원번호     

In [9]:
print("***** Member Data EDA *****")
basic_eda(df_member, "Member Data")

***** Member Data EDA *****
========== Member Data ==========
Shape: (12540, 10)
Columns: ['Unnamed: 0', '회원번호', '회원상태', '성별', '나이', '등록카드', '결혼', '구독여부', '주소지', '세부주소지']

[INFO]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12540 entries, 0 to 12539
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12540 non-null  int64 
 1   회원번호        12540 non-null  int64 
 2   회원상태        12539 non-null  object
 3   성별          12539 non-null  object
 4   나이          12540 non-null  int64 
 5   등록카드        12525 non-null  object
 6   결혼          9823 non-null   object
 7   구독여부        10194 non-null  object
 8   주소지         12537 non-null  object
 9   세부주소지       12531 non-null  object
dtypes: int64(3), object(7)
memory usage: 979.8+ KB
None

[Missing Values]
Unnamed: 0       0
회원번호             0
회원상태             1
성별               1
나이               0
등록카드            15
결혼            2717
구독여부          2346
주소지  

In [10]:
print("***** Product Data EDA *****")
basic_eda(df_product, "Product Data")

***** Product Data EDA *****
========== Product Data ==========
Shape: (2549, 5)
Columns: ['제품번호', '물품명', '물품대분류', '물품중분류', '상품중량']

[INFO]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품번호    2549 non-null   object
 1   물품명     2549 non-null   object
 2   물품대분류   2549 non-null   object
 3   물품중분류   2549 non-null   object
 4   상품중량    2330 non-null   object
dtypes: object(5)
memory usage: 99.7+ KB
None

[Missing Values]
제품번호       0
물품명        0
물품대분류      0
물품중분류      0
상품중량     219
dtype: int64

[Descriptive Statistics]
              제품번호            물품명  물품대분류 물품중분류  상품중량
count         2549           2549   2549  2549  2330
unique        2549           1917     59   384   952
top     100021V2_0  오가닉코튼 여성용 레깅스  식기/편백  냉동생선  300g
freq             1             11    203    64   146

[Head]
         제품번호      물품명  물품대분류   물품중분류       상품중량
0  100021V2_

***재구매율***

In [13]:
# 1. '주문일시' 열을 날짜 형식으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'])

# 2. 각 주문마다 구매횟수를 1로 표시 (새로운 열 추가)
df_sales['구매횟수'] = 1

# 3. 회원번호와 제품번호별 구매 횟수를 pivot_table을 이용해 집계
customer_order_count = df_sales.pivot_table(
    index=['회원번호', '제품번호'],
    values='구매횟수',
    aggfunc='sum'
)

# 4. 각 회원(고객)별로 동일 제품을 2회 이상 구매한 경우가 있는지 확인
#    (groupby의 level='회원번호'를 이용하여 그룹별로 판단)
repurchase_flag = customer_order_count.groupby(level='회원번호')['구매횟수'].apply(lambda s: (s > 1).any())

# 5. 재구매 고객 수와 전체 고객 수 계산
repurchase_customers = repurchase_flag.sum()   # True의 개수 = 재구매 고객 수
total_customers = repurchase_flag.shape[0]

# 6. 재구매율 계산 (백분율)
repurchase_rate = repurchase_customers / total_customers * 100

# 결과 출력
print(f"전체 고객 수: {total_customers}")
print(f"재구매 고객 수: {repurchase_customers}")
print(f"재구매율: {repurchase_rate:.2f}%")


전체 고객 수: 12540
재구매 고객 수: 7997
재구매율: 63.77%


***고객 이탈률***

In [14]:
# 1. 전체 고객 수 (회원번호 기준)
total_customers = df_member['회원번호'].nunique()

# 2. 활동 고객 수: 회원상태가 '정상회원'인 경우
active_customers = df_member[df_member['회원상태'] == '정상회원']['회원번호'].nunique()

# 3. 이탈 고객 수: '정상회원'이 아닌 고객을 이탈 고객으로 판단
churned_customers = total_customers - active_customers

# 4. 고객 이탈률 계산 (백분율)
churn_rate = (churned_customers / total_customers) * 100

print("총 고객 수:", total_customers)
print("이탈 고객 수:", churned_customers)
print(f"고객 이탈률: {churn_rate:.2f}%")

총 고객 수: 12540
이탈 고객 수: 160
고객 이탈률: 1.28%


***배송 지연율***

In [15]:
# 1. 배송 관련 날짜 컬럼을 datetime 형식으로 변환 (에러 발생 시 NaT 처리)
df_sales['배송시작일'] = pd.to_datetime(df_sales['배송시작일'], errors='coerce')
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')

# 2. 배송 완료 정보를 가진 유효 주문만 선택 (취소 주문 및 날짜 미입력 주문 제외)
valid_deliveries = df_sales.dropna(subset=['배송시작일', '배송완료일']).copy()

# 3. 배송 기간(일수) 계산: 배송완료일 - 배송시작일
valid_deliveries['배송기간'] = (valid_deliveries['배송완료일'] - valid_deliveries['배송시작일']).dt.days

# 4. 배송 지연 기준 설정: 예시에서는 배송기간이 1일을 초과하면 지연으로 판단
threshold_days = 1

# 5. 지연 배송 건수 산출: 배송기간이 기준 초과인 주문
delayed_deliveries = valid_deliveries[valid_deliveries['배송기간'] > threshold_days]

# 6. 배송 지연율 계산
delay_rate = (len(delayed_deliveries) / len(valid_deliveries)) * 100

# 결과 출력
print("전체 배송 건수:", len(valid_deliveries))
print("지연 배송 건수:", len(delayed_deliveries))
print(f"배송 지연율: {delay_rate:.2f}%")

전체 배송 건수: 637476
지연 배송 건수: 97067
배송 지연율: 15.23%
